In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [2]:
df = pd.read_csv('Resources/cleaned_data.csv')
df = df.drop(['season','player','team','games_started'],axis=1)
df.head()

,position,age,experience,games,minutes_played,fieldgoals,fieldgoals_attemped,three_points_made,three_point_attempts,two_point_made,...,offensive_rebounds,defensive_rebounds,total_rebounds,assists,steals,blocks,turnovers,personal_fouls,points,playoffs
0,PF,25.0,7,25.0,736.0,128.0,293.0,42.0,112.0,86.0,...,39.0,127.0,166.0,105.0,16.0,20.0,67.0,49.0,364.0,False
1,PF,25.0,7,25.0,648.0,103.0,206.0,17.0,64.0,86.0,...,38.0,80.0,118.0,56.0,17.0,14.0,30.0,40.0,254.0,False
2,PG,24.0,3,66.0,1176.0,170.0,436.0,67.0,182.0,103.0,...,15.0,74.0,89.0,123.0,46.0,13.0,66.0,94.0,475.0,False
3,SF,21.0,1,46.0,669.0,78.0,178.0,40.0,108.0,38.0,...,28.0,99.0,127.0,23.0,15.0,9.0,23.0,87.0,218.0,False
4,SF,27.0,4,24.0,355.0,57.0,116.0,18.0,43.0,39.0,...,7.0,55.0,62.0,19.0,10.0,9.0,19.0,34.0,160.0,False


In [3]:
le = LabelEncoder()

X = df.drop('playoffs', axis=1)
X = pd.get_dummies(X)

# Create our target
y = df['playoffs'].ravel()
#y.describe()

In [4]:
X.describe()

,age,experience,games,minutes_played,fieldgoals,fieldgoals_attemped,three_points_made,three_point_attempts,two_point_made,two_point_attempts,...,steals,blocks,turnovers,personal_fouls,points,position_C,position_PF,position_PG,position_SF,position_SG
count,15074.000000,15074.000000,15074.000000,15074.000000,15074.000000,15074.000000,15074.000000,15074.000000,15074.000000,15074.000000,...,15074.000000,15074.000000,15074.000000,15074.000000,15074.000000,15074.000000,15074.000000,15074.000000,15074.000000,15074.000000
mean,26.688802,5.381651,48.657423,1148.298992,183.993499,397.938371,25.787780,73.247778,158.205718,324.690593,...,37.986400,24.092610,70.275441,104.977312,485.772854,0.202202,0.210362,0.196365,0.188072,0.202999
std,4.071894,3.905992,26.916440,923.949816,181.469290,378.967820,41.942964,111.334306,165.980396,329.881353,...,37.440227,35.858016,66.158490,79.988249,483.190862,0.401656,0.407579,0.397261,0.390783,0.402245
min,18.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,24.000000,2.000000,24.000000,293.250000,34.000000,82.000000,0.000000,2.000000,27.000000,61.000000,...,8.000000,3.000000,16.000000,32.000000,90.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,26.000000,4.000000,53.000000,967.000000,127.500000,285.000000,4.000000,16.000000,102.000000,215.500000,...,27.000000,11.000000,52.000000,95.000000,333.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,29.000000,8.000000,75.000000,1881.750000,285.750000,619.000000,35.000000,104.000000,239.000000,490.000000,...,57.000000,29.000000,107.000000,165.000000,753.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,44.000000,22.000000,82.000000,3485.000000,1098.000000,2279.000000,378.000000,1028.000000,1086.000000,2213.000000,...,263.000000,456.000000,464.000000,371.000000,3041.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [5]:
Counter(y)

Counter({False: 6771, True: 8303})

In [6]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({True: 6208, False: 5097})

# Balanced Random Forest

In [7]:
from sklearn.preprocessing import StandardScaler
#from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Create a random forest classifier.
rf_model = BalancedRandomForestClassifier(n_estimators=128, random_state=1) 

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)
predictions = rf_model.predict(X_test_scaled)

#Counter(predictions)

In [8]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

y_pred = rf_model.predict(X_test)
confusion_matrix(y_test, y_pred)

#print(cm)
# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,1067,607
Actual 1,859,1236


In [9]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

print(acc_score)

0.6110374104537013


In [10]:
# Print the imbalanced classification report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

       False       0.55      0.64      0.59      1674
        True       0.67      0.59      0.63      2095

    accuracy                           0.61      3769
   macro avg       0.61      0.61      0.61      3769
weighted avg       0.62      0.61      0.61      3769



In [11]:
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_
importances
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.06955312376328904, 'games'),
 (0.05967978174429682, 'minutes_played'),
 (0.05293002061765614, 'personal_fouls'),
 (0.04873122378484514, 'fieldgoals_attemped'),
 (0.04832111460493006, 'turnovers'),
 (0.048005521155785796, 'total_rebounds'),
 (0.04780215842947005, 'assists'),
 (0.047376124596301056, 'steals'),
 (0.04684801087549832, 'two_point_attempts'),
 (0.04660973766815497, 'defensive_rebounds'),
 (0.04558933268700156, 'points'),
 (0.04478477589774435, 'freethrows_attempted'),
 (0.0443105938283577, 'offensive_rebounds'),
 (0.04397773931699399, 'age'),
 (0.04318757199046788, 'freethrows_made'),
 (0.04233031547948923, 'experience'),
 (0.041849460756640935, 'two_point_made'),
 (0.04133395856181004, 'blocks'),
 (0.04111656882801677, 'fieldgoals'),
 (0.03961597079382765, 'three_point_attempts'),
 (0.02931176292685244, 'three_points_made'),
 (0.006186206634347808, 'position_SF'),
 (0.005952596409862791, 'position_PF'),
 (0.00556666550741807, 'position_SG'),
 (0.004698586157244735, 'pos

# Easy Ensemble AdaBoost Classifier

In [12]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier

# Create a easy ensemble classifier
ada_model = EasyEnsembleClassifier(n_estimators=128, random_state=1) 

# Fitting the model
ada_model = ada_model.fit(X_train_scaled, y_train)
predictions = ada_model.predict(X_test_scaled)

In [13]:
# Calculated the balanced accuracy score
y_pred2 = ada_model.predict(X_test)
confusion_matrix(y_test, y_pred)

print(cm)

[[1067  607]
 [ 859 1236]]


In [14]:
# Display the confusion matrix
acc_score = accuracy_score(y_test, predictions)

print(acc_score)

0.5935261342531175


In [15]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

      False       0.53      0.65      0.55      0.59      0.60      0.36      1674
       True       0.66      0.55      0.65      0.60      0.60      0.35      2095

avg / total       0.61      0.59      0.61      0.59      0.60      0.36      3769

